# Top 10 arrivals airports in 2013

rrival airport is the cilumn arr_port. Is is the IATA code for the airport. 

To get the total number of passedngers for an airport, you can sum the column pax, grouping by arr_port

Note tha rere is negative pax (Cancelations). Puedes sumar incluyendo negativos para incluir cancelaciones

El formmato BZ2 es el que mejor va en sistemas de ficheros como HADOOP. Hadoop particiona ficheros en bloques y esa gestión de bloques se hace mejor en BZ2.

Para ficheros grandes: BZ2.

Para ficheros pequeños:ZIP

## Pasos

* Preparar muestras
* Familiarizarse con datos
* Seleccionar columnas de interés.
* Gestionar los NaN.
* Hace una plan de acción
* Desarrollar código que funciona con una nuestra
* Ajusta código para que funcione con Big Data
* Probar ese código con la muestra
* Probar ese código con big data

### 1 ) Nos familiarizamos con datos

In [6]:
cd ~/Data/challenge

/home/dsc/Data/challenge


In [7]:
! ls -alt

total 1018748
drwxr-x--- 2 dsc dsc      4096 ene 25 17:58 .
-rwxrwxrwx 1 dsc dsc    244720 ene 25 17:20 searches_sample.csv.bz2
-rwxrwxrwx 1 dsc dsc   4232732 ene 25 17:19 bookings_sample.csv
-rwxrwxrwx 1 dsc dsc    535893 ene 25 17:19 bookings_sample.csv.bz2
-rwxr-x--- 1 dsc dsc 483188920 mar 13  2018 searches.csv.bz2
-rwxr-x--- 1 dsc dsc 554970628 mar 13  2018 bookings.csv.bz2
drwxr-x--- 7 dsc dsc      4096 mar 13  2018 ..


In [8]:
import pandas as pd

In [9]:
#DONT DO THIS!!!
# b = pd.read_csv("bokkings.csv.bz2", sep="^")

In [11]:
bookings = pd.read_csv('bookings_sample.csv.bz2', sep='^')

_Nota_: esto se puede hacer, aunque es algo más lento que lo de BZ2File

Otra opción habría sido haber liedo 10000 líneas del fichero grande:

In [13]:
bookings = pd.read_csv('bookings.csv.bz2', sep='^', nrows=10000)

In [14]:
type(bookings)

pandas.core.frame.DataFrame

In [15]:
bookings.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


_Nota_: son 38 columnas y no las puedo ver. Lo resuelvo así:

In [17]:
pd.set_option('display.max_columns', None)

In [18]:
bookings.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


### Sample es mejor, porque coge filas aleatoriamente

In [19]:
bookings.sample(5)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
8477,2013-03-11 00:00:00,1V,US,1747eae623a529e7995d4e0cc0a74334,839b98631801dad4de8d8c048dad8285,eeb207809a52c9ec679ee944634f6a2f,2013-03-11 00:00:00,750,0,CMH,CMH,US,PHL,PHL,US,CMH,CMH,US,CMH,CMH,US,PHL,PHL,US,CMHPHL,CMHPHL,USUS,0,CMHPHL,FD,N,Y,2013-03-15 07:08:00,2013-03-15 08:41:57,1,2013,3,NULL
4430,2013-03-07 00:00:00,1S,CA,5eff7f2d504b03426a7ba7d37075e6c9,cf400ac1e069f9e5f7f8feb012f8cc13,32019a770d09efab37ceca40fb15dcfa,2013-03-07 00:00:00,3009,0,YYZ,YTO,CA,GSP,GSP,US,YYZ,YTO,CA,CLT,CLT,US,YYZ,YTO,CA,CLTYYZ,CLTYTO,CAUS,1,CLTYYZ,KM,M,Y,2013-03-15 17:05:00,2013-03-15 18:59:51,1,2013,3,NULL
8031,2013-03-21 00:00:00,1V,US,4d5998e7917ae5dbf539f9c2032aee98,6083d06066d003bd577b5a063f561b07,b38a9ac81beaacbfe056d9258abd54f7,2013-03-21 00:00:00,5087,0,LAX,LAX,US,BOS,BOS,US,LAX,LAX,US,BOS,BOS,US,PDX,PDX,US,BOSPDX,BOSPDX,USUS,0,BOSPDX,LY,U,Y,2013-04-10 18:45:00,2013-04-10 21:20:16,1,2013,3,NULL
7603,2013-03-29 00:00:00,1V,JP,75ad6fea47c9fd4ba9fb7a9aef158288,df227b2780c913977230277423708816,f8016fd1eb24d33d96800f98a91ee32e,2013-03-29 00:00:00,6056,0,NRT,TYO,JP,ICN,SEL,KR,NRT,TYO,JP,ICN,SEL,KR,NRT,TYO,JP,ICNNRT,SELTYO,JPKR,1,ICNNRT,YK,K,Y,2013-06-10 15:10:00,2013-06-10 17:26:40,1,2013,3,NULL
5337,2013-03-31 00:00:00,1P,US,83ccf07a55606976f78e63fcc805f58c,1490c3c5f95f04fa076fe15c775a80a9,4dff4efdb5f2ec5ee82384a81a08b5c3,2013-03-25 00:00:00,13876,0,DEN,DEN,US,BZE,BZE,BZ,DEN,DEN,US,BZE,BZE,BZ,DEN,DEN,US,BZEDEN,BZEDEN,BZUS,1,BZEMIADEN,KK,Q,Y,2013-08-15 14:15:00,2013-08-15 22:36:54,2,2013,3,NULL


Curiosidades sobre las columnas:
- act_date no incluye fecha y hora
- source es el que lo ha gestionado (Amadeus, uno de la competencia de Amadeus, etc.)
- pos_ctry: ciudad de venta
- pos_iata, pos_oid y rloc son hashes, por un tema de privacidad. Por_oid por ejemplo es la agencia que vende. 
- rloc: el localizador de la reserva. 
- dep_port: aeuropuerto de salida del vuelo individualmente
- arr: análogo para llada
- si voy a madrid a franfurk y vuelvo a madrid, el boarding port seŕia Madrid y el OffPort sería Franfurt. Así distinguimos columnas 
- Si hago Madrid- Paris-Franfurt, pero en PAris estoy una hora, sólo metemos una fila. La columna route sería MADPRAFRA. Si estoy más de 24 horas ta lo tratan como filas independienes
- Carrier: Iberiametc...
- cab_class: clase.
- brd_timy offtime es fecha de embarque y salida de vuelo

TODO ESTO LO EXPLICA PORQUE DICE QUE SI NO ENTIENDES POR EJEMPLO LO DEL BOARDING PORT, TE PUEDEN PEDIR COSAS, TÚ CREES QUE LO HAS HECHO BIEN Y EN REALIDAD ESTÁS DANDO DATOS, POR EJEMPLO, DUPLICADOS

### 2) Seleccionamos las columnas de interés

In [27]:
bookings[ ["arr_port", "pax", "year"]].sample(5)

,arr_port,pax,year
2880,LAX,1,2013
4970,SDF,1,2013
1208,YYZ,1,2013
769,LAS,-2,2013
5023,BHM,1,2013
